In [1]:
# Six Small Algorithm Projects
# 1.     Trees
# 1.5    Drawings

In [2]:
import tkinter as tk

In [3]:
#Create a N-ary Node Class
class NaryNode:
    indent = '  '
    node_radius = 10
    x_spacing = 20
    y_spacing = 20

    def __init__(self, value):
        self.value = value
        self.children = []

        # Initialize drawing parameters.
        self.center = (0, 0)
        self.subtree_bounds = (
            self.center[0] - NaryNode.node_radius, 
            self.center[1] - NaryNode.node_radius, 
            self.center[0] + NaryNode.node_radius, 
            self.center[1] + NaryNode.node_radius)

    def add_child(self, node):
        self.children.append(node)
        node.parent = self

    def __str__(self, level=0):
        '''Recursively create a string representation of this node's subtree.
        Display this value indented, followed by the child values indented one more level.
        End in a newline.'''
        result = level * NaryNode.indent + f'{self.value}:\n'
        for child in self.children:
            result += child.__str__(level + 1)
        return result

    def find_node(self, target):
        '''Recursively search the subtree rooted at this node for the target value.
        If found, return the node containing the target value, otherwise return None.'''
        if self.value == target:
            return self
        
        for child in self.children:
            result = child.find_node(target)
            if result != None:
                return result
        
        return None
    
    def traverse_preorder(self):
        '''Traverse the subtree rooted at this node in preorder.
        Return a list of each node's value as it is visited.'''
        preorder_list = [self.value]
        for child in self.children:
            preorder_list += child.traverse_preorder()
        return preorder_list
            
    def traverse_postorder(self):
        '''Traverse the subtree rooted at this node in postorder.
        Return a list of each node's value as it is visited.'''
        postorder_list = []
        for child in self.children:
            postorder_list += child.traverse_postorder()
        postorder_list.append(self.value)
        return postorder_list
        
    def traverse_breadth_first(self):
        '''Traverse the subtree rooted at this node in breadth-first order.
        Yield each node's value as it is visited.'''
        queue = [self]
        result = []
        while queue:
            node = queue.pop(0)
            result.append(node.value)
            queue.extend(node.children)
        return result
    
    def arrange_subtree(self, xmin, ymin):
        '''Position the node's subtree.'''
        # Calculate cy, the Y coordinate for this node.
        # This doesn't depend on the children.
        cy = ymin + NaryNode.node_radius

        # If the node has no children, just place it here and return.
        if len(self.children) == 0:
            cx = xmin + NaryNode.node_radius
            self.center = (cx, cy)
            xmax = xmin + 2 * NaryNode.node_radius
            ymax = ymin + 2 * NaryNode.node_radius
            self.subtree_bounds = (xmin, ymin, xmax, ymax)
            return

        # Set child_xmin and child_ymin to the
        # start position for child subtrees.
        child_xmin = xmin
        child_ymin = ymin + 2 * NaryNode.node_radius + NaryNode.y_spacing

        # Set ymax equal to the largest Y position used.
        ymax = ymin + 2 * NaryNode.node_radius

        # Position the child subtrees.
        for child in self.children:
            # Position this child subtree.
            child.arrange_subtree(child_xmin, child_ymin)

            # Update child_xmin to allow room for the subtree
            # and space between the subtrees.
            child_xmin = child.subtree_bounds[2] + NaryNode.x_spacing

            # Update the subtree bottom ymax.
            if ymax < child.subtree_bounds[3]:
                ymax = child.subtree_bounds[3]

        # Set xmax equal to child_xmin minus the horizontal
        # spacing we added after the last subtree.
        xmax = child_xmin - NaryNode.x_spacing

        # Use xmin, ymin, xmax, and ymax to set our subtree bounds.
        self.subtree_bounds = (xmin, ymin, xmax, ymax)

        # Center this node over the subtree bounds.
        cx = (self.subtree_bounds[0] + self.subtree_bounds[2]) / 2
        cy = ymin + NaryNode.node_radius
        self.center = (cx, cy)

    def draw_subtree_links(self, canvas):
        ''' Draw the subtree's links.'''
        # If we have exactly one child, just draw to it.
        if len(self.children) == 1:
            child = self.children[0]
            canvas.create_line(
                self.center[0], self.center[1],
                child.center[0], child.center[1],
                fill='green')

        # Else if we have more than one child,
        # draw vertical and horizontal branches.
        elif len(self.children) > 0:
            # Find the Y coordinate of the center
            # halfway to the children.
            ymid = (self.center[1] + self.children[0].center[1]) / 2
            # Draw the vertical line to the center line.
            canvas.create_line(
                self.center[0], self.center[1],
                self.center[0], ymid,
                fill='green')

            # Draw the horizontal center line over the children.
            last_child = len(self.children) - 1
            canvas.create_line(
                self.children[0].center[0], ymid,
                self.children[last_child].center[0], ymid,
                fill='green')

            # Draw lines from the center line to the children.
            for child in self.children:
                canvas.create_line(
                    child.center[0], ymid,
                    child.center[0], child.center[1],
                    fill='green')

        # Recursively draw child subtree links.
        for child in self.children:
            child.draw_subtree_links(canvas)

        # Outline the subtree for debugging.
        #canvas.create_rectangle(self.subtree_bounds, fill='', outline='red')

    def draw_subtree_nodes(self, canvas):
        ''' Draw the subtree's nodes.'''
        # Draw the node.
        x0 = self.center[0] - NaryNode.node_radius
        y0 = self.center[1] - NaryNode.node_radius
        x1 = self.center[0] + NaryNode.node_radius
        y1 = self.center[1] + NaryNode.node_radius
        canvas.create_oval(x0, y0, x1, y1, fill='white', outline='black')
        canvas.create_text(self.center, text=self.value, fill='red')

        # Draw the descendants' nodes.
        for child in self.children:
            child.draw_subtree_nodes(canvas)

    def arrange_and_draw_subtree(self, canvas, xmin, ymin):
        # Position the tree.
        self.arrange_subtree(xmin, ymin)

        # Draw the links.
        self.draw_subtree_links(canvas)

        # Draw the nodes.
        self.draw_subtree_nodes(canvas)

In [4]:
# Create a test tree that looks like this:

#         A
#         |
#     +---+---+
#     B   C   D
#     |       |
#    +-+      +
#    E F      G
#    |        |
#    +      +-+-+
#    H      I J K

a = NaryNode('A')
b = NaryNode('B')
c = NaryNode('C')
d = NaryNode('D')
e = NaryNode('E')
f = NaryNode('F')
g = NaryNode('G')
h = NaryNode('H')
i = NaryNode('I')
j = NaryNode('J')
k = NaryNode('K')

a.add_child(b)
a.add_child(c)
a.add_child(d)
b.add_child(e)
b.add_child(f)
d.add_child(g)
e.add_child(h)
g.add_child(i)
g.add_child(j)
g.add_child(k)

In [5]:
def kill_callback():
    """A callback to destroy the tkinter window."""
    window.destroy()

In [6]:
# Make the tkinter window.
window = tk.Tk()
window.title("nary_node5")
window.protocol("WM_DELETE_WINDOW", kill_callback)
window.geometry("260x180")

canvas = tk.Canvas(window, bg="white", borderwidth=2, relief=tk.SUNKEN)
canvas.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

# Draw the tree.
a.arrange_and_draw_subtree(canvas, 10, 10)

window.focus_force()
window.mainloop()